<strong>Date :</strong> Créé le 03 Avril 2021| Mis à jour le 10 Avril 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo SACCAREAU & Théo VEDIS

<strong>(2-5)create_features_others
      
Description :</strong> A travers ce Notebook, nous détaillerons notre méthode pour créer toutes sortes de features que nous jugeons pertinentes pour prédire le score d'un commentaire mais qui ne correspondaient pas à des features liées au texte ou aux réseaux.

Temps d'exécution du Notebook : environ  5min.

# Installer / Télécharger / Importer des librairies

In [1]:
# Librairies usuelles 
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm 
import json

# Librairie pour gérer le format de date UTC 
from datetime import datetime

# Chemin 

In [2]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

# Chargement des données d'entrée

Nous avons besoin du fichier créé par le Notebook précédent. 

In [3]:
# Temps exécution : 2min 
with open( pathFile + 'df_features_network2.json') as json_data:
    data_dict = json.load(json_data)

# On transforme le dictionnaire en DataFrame 
# Temps exécution : 1min30
df = pd.DataFrame(data_dict)

# On libère de la RAM en supprimant la variable data_dict qui ne nous servira plus
del data_dict

In [4]:
df.head()

,created_utc,ups,link_id,name,author,parent_id,is_parent_comment,is_author_deleted,is_body_deleted,bot_comment,length_comment_chars_before_NLP,length_comment_chars_after_NLP,length_comment_words,nb_stopwords,sentiment,sentiment_child,depth,nb_direct_resp,nb_total_resp,author_centrality_degree,author_centrality_ev,author_prestige_degree
0,1430438400,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,1,0,0,0,119,59,8,14,neg,0,2,0,0,0.000138,1.312352e-03,0.000072
1,1430438400,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,0,0,0,0,48,13,2,7,neg,0,2,0,0,0.000005,2.167334e-19,0.000000
2,1430438400,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,t1_cqu80zb,1,0,0,0,4,4,1,0,neu,0,2,0,0,0.000126,1.712427e-03,0.000051
3,1430438401,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,t1_cqtdj4m,1,0,0,0,54,15,3,12,neg,0,2,0,0,0.000000,2.167334e-19,0.000000
4,1430438401,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,t1_cquc4rc,1,0,0,0,241,164,24,20,pos,0,2,1,2,0.000004,5.092290e-05,0.000002


Nous avons également besoin de récupérer les informations que nous avons scrappée concernant les topics/posts. 

In [5]:
# On récupère également les informations scrapées sur les topics.
# Temps exécution : 40s
df2 = pd.read_json(pathFile + "data_post.json").T 

In [6]:
df2.head()

,score,up_vote_ratio,author,title,time
t3_36sia6,2,1,HereOrToGo,What is the funniest YouTube video you've seen...,1432237175
t3_36njmx,3,1,healyemojicarol,what's the most important thing you look for i...,1432145117
t3_364lbx,3,1,Akwardswan,"Dear reddit, whats the weirdest thing you foun...",1431742334
t3_34qa1x,3,0.8,None,What is your favorite video game of all time a...,1430671104
t3_36rw77,0,0.4,None,If someone offered you a very large sum of mon...,1432227976


Nous avions aussi besoin des informations scrappées concernant les auteurs.

In [7]:
# On récupère aussi les informations scrapées sur les auteurs.
# Temps exécution : 
with open( pathFile + 'data_author.json') as json_data:
    data_dict = json.load(json_data)

# On transforme le dictionnaire en DataFrame 
df3 = pd.DataFrame(data_dict).T

# On libère de la RAM en supprimant la vari ble data_dict qui ne nous servira plus
del data_dict

In [8]:
df3.head()

,has_verified_email,comment_karma,time
shawa666,True,56448,1274134587
enolafaye,True,10260,1406868017
AshKals,True,21516,1319661313
ucfnate,True,8135,1323926375
JustThePit,False,8265,1374728085


# (1) Features sur les dates 
A partir de la colonne `created_utc`, qui correspond à la date au format UTC (cf Notebook Obs & Nettoyage), nous allons en déduire trois nouvelles features : 
- L'heure (heure/minutes/secondes) du commentaire.   
- Le temps de réponse entre le commentaire et son parent.
- La temps de réponse entre le commentaire et le post.

## 1-1 Heure du commentaire
Nous estimons que l'heure d'un commentaire peut avoir une influence sur le score obtenu. Effectivement, il y a certain crénaux horaires où il y a moins d'utilisateurs connectés à Reddit (et lorsque l'affluence est de nouveau importante, le commentaire se retrouve en bas d'une discussion ou est moins visible). La visibilité du commentaire est donc moins importante, ce qui peut impacter négativement son score.  

In [9]:
# On transforme la date au format UTC en heure de publication
df['hour'] = df['created_utc'].apply(lambda x:  int(
    datetime.utcfromtimestamp(x).strftime("%H")))

# On transforme l'heure de publication en moment de la journée :
# matin (7h-12h), après-midi (13h-19h), soir (20h-23h) ou nuit(0h-6h).
df['hour'] = ['morning' if h >= 7 and h <= 12 else 'afternoon' if h >= 13 and h <=
              19 else 'evening' if h >= 20 and h <= 23 else 'night' for h in df['hour']]

In [10]:
# Résultat
df['hour'][-10:]

4234960    evening
4234961    evening
4234962    evening
4234963    evening
4234964    evening
4234965    evening
4234966    evening
4234967    evening
4234968    evening
4234969    evening
Name: hour, dtype: object

C'est normal qu'il n'y ait que des "soirs" à la fin du jeu de données car on a vu dans l'observation des données que ces données correspondaient aux commentaires Reddit du 1er Mai 2015 0h00 au 31 Mai 2015 23h59. 

## 1-2 Temps de réponse commentaire-parent 
Cette nouvelle variable nous permet de connaitre le temps de réponse d'un commentaire vis à vis de son parent. Pour nous, plus le temps de réponse est important plus le score risque d'être faible. En effet, un commentaire qui répond à un commentaire ou sujet datant de plusieurs jours, aura forcément moins de visibilité qu'un commentaire qui a répondu immédiatement. 

In [11]:
# On crée un ensemble contenant les noms de chaque commentaire. 
# On l'utilisera pour savoir si le parent d'un commentaire fait partie de notre 
# base de données (et donc qu'on peut connaitre son heure de mise en ligne) ou pas 
ens_name = set(df['name'])

# On rajoute les topics 
ens_name.update(set(df2.index)) # on aurait pu utiliser set(df['link_id']) les 2 sont équivalents

In [12]:
# Dictionnaire qui associe commentaire-date
dict_comm_date = dict([(name, date)
                       for name, date in zip(tqdm(df['name']), df['created_utc'])])

# Grâce aux informations scrappées, on rajoute la date des topics
dict_comm_date.update(
    dict([(topic, date) for topic, date in zip(tqdm(df2.index), df2['time'])]))

100%|██████████| 148848/148848 [00:00<00:00, 1195426.27it/s]


In [13]:
# On crée une liste qui contient la date de mise en ligne du parent
parent_times = [dict_comm_date[parent]
                if parent in ens_name else None for parent in tqdm(df['parent_id'])]

# On fait la soustraction entre la date de mise en ligne du parent et celle du
# commentaire actuel pour connaitre le temps de réponse commentaire-parent.
df['resp_time_parent'] = [time - parent_time if parent_time is not None else None for parent_time,
                          time in zip(tqdm(parent_times), df['created_utc'])]

100%|██████████| 4234970/4234970 [00:02<00:00, 1794902.12it/s]


In [14]:
# Résultat 
df['resp_time_parent'][-10:]

4234960     2412.0
4234961    15267.0
4234962     8739.0
4234963    31793.0
4234964     4194.0
4234965    44849.0
4234966    44849.0
4234967     1952.0
4234968    41055.0
4234969     1807.0
Name: resp_time_parent, dtype: float64

Remarque : Le résultat est en nombre de secondes car on a soustrait deux formats UTC (secondes écoulées depuis le 1er janvier 1970 00:00:00). 

## 1-3 Temps de réponse commentaire-topic 
Quasiement identique à la feature précédente sauf qu'ici on regarde le temps de réponse entre le commentaire et le sujet (et non le parent). Dans certains cas, les deux informations peuvent être identiques (si le commentaire est directement une réponse à un topic), mais d'en d'autres situations, ce sera une information complémentaire. <br>
Comme précédent, nous estimons que plus le temps de réponse est important plus le score risque d'être faible. En effet, un commentaire qui répond à un sujet datant de plusieurs jours, aura forcément moins de visibilité qu'un commentaire qui a répondu immédiatement. 

In [15]:
# On crée un ensemble contenant les noms de chaque topic. 
# On l'utilisera pour savoir si le topic d'un commentaire fait partie des informations 
# scrappées (et donc qu'on peut connaitre son heure de mise en ligne) ou pas. 
# Normalement, il ne devrait pas y avoir d'oublis donc cette précaution pour éviter 
# d'éventuelles erreurs peut être omise.  
ens_topic = set(df2.index) # on aurait pu utiliser set(df['link_id'])

In [16]:
# Dictionnaire qui associe topic-date
dict_topic_date = dict([(topic,date) for topic, date in zip(tqdm(df2.index), df2['time'])])

100%|██████████| 148848/148848 [00:00<00:00, 375921.88it/s]


In [17]:
# On crée une liste qui contient la date de mise en ligne du topic
topic_times = [dict_topic_date[topic]
               if topic in ens_topic else None for topic in tqdm(df['link_id'])]

# On fait la soustraction entre la date de mise en ligne du topic et celle du
# commentaire actuel pour connaitre le temps de réponse commentaire-topic.
df['resp_time_topic'] = [time - topic_time if topic_time is not None else None for topic_time,
                         time in zip(tqdm(topic_times), df['created_utc'])]

100%|██████████| 4234970/4234970 [00:02<00:00, 2018097.07it/s]


In [18]:
# Résultat
df['resp_time_topic'][:10]

0     22894
1     13205
2     20339
3    109679
4     22895
5     11879
6     13207
7       698
8       109
9       549
Name: resp_time_topic, dtype: int64

Désormais qu'on a créé toutes les features souhaitées à partir de la colonne `created_utc`, on peut la supprimer. 

In [19]:
# Suppression colonne 'created_utc'
df = df.drop(columns=['created_utc'])

# (2) Features sur la popularité du sujet 
Dans cette partie, nous allons extraire des features liées à la popularité du sujet. Nous pensons que cela peut être une bonne indication pour prédire le score d'un commentaire. Les features qui seront traités ici sont : 
- le score du topic dans lequel se trouve le commentaire et éventuellement le score du commentaire au-dessus (s'il est connu, c'est-à-dire, s'il se trouve dans la base d'apprentissage). 
- le nombre de commentaires du topic et le nombre de commentaire du parent.
- le rapport du nombre de votes positifs sur le nombre de votes total pour un sujet/post 

## 2-1 Score du topic et du parent
Le fait de connaitre cette information peut aider, selon nous, à prédire le score d'un commentaire. Effectivement, si le topic et/ou le parent de notre commentaire ont un score important, notre commentaire a plus de chance d'avoir un score plus élevé qu'un commentaire qui a un topic et son parent avec un petit score. <br>
Bien évidemment, il peut y avoir des exceptions, par exemple, un commentaire qui répond à un sujet polémique (ayant reçu majoritairement des unlikes plutôt que des likes) et qui le contredit se verra recevoir beaucoup de likes de la part des utilisateurs qui ne sont pas d'accord avec le post initial.


Remarque : Si le parent du commentaire est un topic (t3) alors les deux variables seront identiques dans ce cas. 

### 2-1-1 Score parent

In [20]:
# Dictionnaire commentaire-score
dict_comm_score = dict([(name, score)
                        for name, score in zip(tqdm(df['name']), df['ups'])])

# Grâce aux informations scrappées, on rajoute le score des topics
dict_comm_score.update(
    dict([(topic, score) for topic, score in zip(tqdm(df2.index), df2['score'])]))

# A partir de ce dictionnaire, on crée une liste qui contient le score du parent
# On utilise l'ensemble 'ens_name' créé précédemment.
df['score_parent'] = [dict_comm_score[parent]
                      if parent in ens_name else None for parent in tqdm(df['parent_id'])]

100%|██████████| 4234970/4234970 [00:04<00:00, 1039939.65it/s]


### 2-1-2 Score topic

In [21]:
# Dictionnaire qui associe topic-score
dict_topic_score = dict([(topic, score)
                         for topic, score in zip(df2.index, df2['score'])])

# On crée une liste qui contient le score pour chaque topic associé à un commentaire
# On utilise l'ensemble 'ens_topic' utilisé dans la partie précédente.
df['score_topic'] = [dict_topic_score[topic]
                     if topic in ens_topic else None for topic in tqdm(df['link_id'])]

100%|██████████| 4234970/4234970 [00:02<00:00, 1718370.38it/s]


## 2-2 Nombre de commentaires pour le topic et/ou le parent
Cette feature peut être vu de façon différente pour son aide à prédire un score.<br> 
En effet, on peut se dire que plus le nombre de commentaires est élevé pour le topic et/ou le parent, plus notre commentaire va se noyer dans la masse de commentaires, et donc difficilement recevoir des likes. Ainsi, plus cette variable serait élevée, plus le score serait faible.<br> 
Cependant, on peut voir la situation d'une autre façon. On peut effectivement considérer que plus le nombre de commentaires est élevé, plus c'est un sujet populaire et donc plus l'audience est élevée. Ce qui signifierait que plus il y a de commentaires, plus notre commentaire a de chance d'avoir plein de likes et donc un score important.
Remarques :
- Ici on souhaite avoir le nombre de commentaire <strong>directs</strong>.
- Si le parent du commentaire est un topic (t3) alors les deux variables seront identiques.

In [22]:
# Grâce à la fonction value_counts, on récupère pour chaque parent (à la fois topic ou commentaire),
# le nombre de fois qu'ils sont les parents d'un commentaire, ce qui correspond au nombre de
# commentaires reçu.
dict_nb_comments = dict(df['parent_id'].value_counts())

# Nombre de commentaires reçu par le parent du commentaire
df['nb_comments_parent'] = df['parent_id'].apply(lambda x: dict_nb_comments[x])

# Nombre de commentaires reçu par le sujet/topic du commentaire
df['nb_comments_topic'] = df['link_id'].apply(
    lambda x: dict_nb_comments[x] if x in dict_nb_comments else 0)

## 2-3 Rapport du nombre de votes positifs sur le nombre de votes total pour un sujet/post 
Pour cette dernière feature, nous utilisons directement l'information scrapée sur Reddit. <br> 
Pour nous, connaitre le rapport nombre de votes positifs reçus sur nombre de votes total pour un sujet est un indicateur sur sa popularité. <br> 
Pour nous un post qui a un ratio de 1 (qui a reçu que des likes et pas de dislikes), est un post apprécié et donc potentiellement ses commentaires le seront aussi.  

In [23]:
# Dictionnaire qui associe topic-ratio
dict_topic_ratio = dict([(topic, ratio)
                         for topic, ratio in zip(df2.index, df2['up_vote_ratio'])])

# On crée une liste qui contient le ratio pour chaque topic associé à un commentaire
# On utilise l'ensemble 'ens_topic' utilisé dans la partie précédente.
df['ratio_topic'] = [dict_topic_ratio[topic]
                     if topic in ens_topic else None for topic in tqdm(df['link_id'])]

100%|██████████| 4234970/4234970 [00:02<00:00, 1701116.76it/s]


# (3) Features sur les auteurs
Dans cette 3e partie, nous nous concentrons sur les auteurs. Cinq nouvelles features seront créées : 
- Le score moyen de l'auteur (pour les commentaires présents dans l'échantillon train).
- Variable binaire permettant de savoir si l'auteur du commentaire est le même auteur que le post. 
- Année de création de son compte
- Vérification du compte 
- Comment karma 

## 3-1 Score moyen de l'auteur 
Le fait d'avoir le score moyen d'un auteur est bien évidement déterminant dans la prédiction du score d'un de ces nouveaux commentaires. Si en général, il obtient des scores élevés, c'est qu'il donne des avis/commentaires pertinents et appréciés par une partie des utilisateurs de Reddit. Ainsi, pour tout nouveau commentaire, il y a plus de chance qu'il obtienne un bon score qu'un auteur qui obtient d'habitude de mauvais scores. 

In [24]:
# On crée un sous DataFrame qui ne contient que les colonnes Author et ups(score)
# et où le score est non null.
df_author_ups = df[df['ups'].notna()][['author', 'ups']]

# On regroupe par auteur, on fait la moyenne des scores et on transforme le tout
# en dictionnaire (où la clé est l'auteur et la valeur son score moyen).
dico_author_score_means = df_author_ups.groupby(
    'author').mean().to_dict()['ups']

# On en déduit la nouvelle feature
df['author_mean_score'] = df['author'].apply(
    lambda x: dico_author_score_means[x] if x in dico_author_score_means else None)

In [25]:
# Regardons le nombre de commentaires qui n'a pas de score moyen pour son auteur. 
100 * sum(df['author_mean_score'].isnull()) / df.shape[0] 

4.332710739391306

Il n'y a donc près 96% des commentaires où l'on connaitra le score moyen de l'auteur, c'est très satisfaisant. 

## 3-2 Même auteur que le topic 
Pour nous, si l'auteur d'un commentaire est le même que l'auteur du topic initial, il y a de bonne chance que le commentaire est un bon score (car il peut apporter une précision, un complément d'informations, etc).
Ce n'est qu'une hypothèse, nous verons lors de l'apprentissage du modèle si cette feature est importante ou pas. <br>
Ce sera donc une variable binaire avec 0 si l'auteur du commentaire est différent de l'auteur du topic, 1 si ce sont les mêmes.

In [26]:
# On remplace les 'None' (str) issus du scraping par des None "python" 
df2['author'] = df2['author'].apply(lambda x : None if x == 'None' else x)

In [27]:
# Dictionnaire qui associe topic-author
dict_topic_author = dict([(topic, author)
                          for topic, author in zip(tqdm(df2.index), df2['author'])])

# On crée une liste qui contient l'auteur de chaque topic
topic_authors = [dict_topic_author[topic]
                 if topic in ens_topic else None for topic in tqdm(df['link_id'])]

# On vérifie si l'auteur du topic est le même que celui du commentaire actuel
df['isSameAuthor'] = [author_comment == author_topic for author_comment,
                      author_topic in zip(tqdm(df['author']), topic_authors)]

100%|██████████| 4234970/4234970 [00:01<00:00, 2191363.71it/s]


In [28]:
sum(df['isSameAuthor'])

98299

Pour 98 299 commentaires (sur 4 millions), l'auteur du commentaire est le même que l'auteur du post initial.

## 3-3 Année de création de son compte 
L'année de création du compte de l'auteur peut éventuellement être une information permettant de déterminer le score de son commentaire. Nous estimons en effet que plus l'auteur est inscrit depuis longtemps, plus il comprend le fonctionnement de Reddit et des astuces pour avoir plus de votes positifs. 


In [29]:
# On crée un sous DataFrame qui ne contient que les colonnes Author et Ups(score) 
# et où le score est non null. 
df_time = df3[df3['time'].notna()]['time']

In [30]:
# Dictionnaire qui associe auteur-date de création
dict_author_date = dict([(author, int(datetime.utcfromtimestamp(date).strftime(
    "%Y"))) for author, date in zip(tqdm(df_time.index), df_time.values)])

100%|██████████| 504821/504821 [00:02<00:00, 239105.64it/s]


In [31]:
# A partir du dictionnaire précédent, on en déduit pour chaque commentaire
# l'année de création de compte pour son auteur.
df['year_creation_author'] = [dict_author_date[author]
                              if author in dict_author_date else None for author in tqdm(df['author'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1113166.88it/s]


In [32]:
# Résultat
df['year_creation_author'][:10]

0    2013.0
1    2012.0
2    2013.0
3    2013.0
4    2013.0
5    2014.0
6    2015.0
7       NaN
8    2014.0
9    2010.0
Name: year_creation_author, dtype: float64

## 3-4 Vérification du compte 
Le fait de savoir si l'auteur a vérifié son email peut nous aider à prédire le score puisque nous pensons que les auteurs qui ne l'ont pas fait sont soit des auteurs "robots" soit des auteurs qui peuvent potentiellement mettre des commentaires polémiques, insultants, etc (et donc avoir des scores faibles voire négatif). 


In [33]:
# Dictionnaire qui associe auteur-vérification de compte
dict_author_verif = dict([(author,verif) for author, verif in zip(df3.index, df3['has_verified_email'])])

# On crée une liste qui contient l'information sur la vérification du compte 
# pour chaque auteur associé à un commentaire.
df['has_verified_email'] = [dict_author_verif[author] if author in dict_author_verif else None for author in tqdm(df['author'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1130178.61it/s]


## 3-5 Comment Karma 
Enfin, la dernière feature mise au point est le "comment karma" d'un auteur.  Le comment karma est une métrique utilisée par Reddit qui correspond à la somme des scores reçus par un auteur. <br> 
Ainsi, plus cette métrique est importante, plus l'auteur a reçu de likes depuis la création de son compte. Il donne alors des avis/commentaires pertinents et appréciés par une partie des utilisateurs de Reddit. Ainsi, pour tout nouveau commentaire, il y a plus de chance qu'il obtienne un bon score qu'un auteur a un comment karma faible. <br> 
Remarque : Cette feature peut sembler redondante avec la moyenne des scores des auteurs, sauf qu'ici le comment karma concerne les scores sur toute période alors que la moyenne des scores des auteurs est basée sur les scores de notre jeu de données, c'est-à-dire sur les scores obtenues en Mai 2015 seulement.  


In [34]:
# Dictionnaire qui associe auteur-comment karma
dict_author_karma = dict(
    [(author, karma) for author, karma in zip(df3.index, df3['comment_karma'])])

# On crée une liste qui contient l'information sur le comment karma pour 
# chaque auteur associé à un commentaire.
df['comment_karma'] = [dict_author_karma[author]
                       if author in dict_author_karma else None for author in tqdm(df['author'])]

100%|██████████| 4234970/4234970 [00:04<00:00, 1024320.18it/s]


# Conclusion 
Après avoir travaillé sur les réseaux puis sur le contenu textuel des commentaires, nous avons dans ce Notebook ajouté des features portant sur les dates, les auteurs ou la popularité du topic. C'est ce qui conclut notre phase sur notre recherche de features pouvant nous aider à prédire le score d'un commentaire posté sur Reddit. 

Nous allons désormais pouvoir passer l'apprentissage d'un modèle prédictif. Pour cela, nous devons stocker le dataframe contenant l'ensemble des features. 

Les colonnes `link_id`, `author` et `parent_id` sont désormais inutiles, nous pouvons les supprimer pour ne garder que les colonnes dédiées aux features et la colonne liée au score (`ups`).  

In [35]:
# Colonnes à supprimer 
columns_to_delete = ['link_id', 'author', 'parent_id']

df = df.drop(columns=columns_to_delete)

In [36]:
df.head()

,ups,name,is_parent_comment,is_author_deleted,is_body_deleted,bot_comment,length_comment_chars_before_NLP,length_comment_chars_after_NLP,length_comment_words,nb_stopwords,sentiment,sentiment_child,depth,nb_direct_resp,nb_total_resp,author_centrality_degree,author_centrality_ev,author_prestige_degree,hour,resp_time_parent,resp_time_topic,score_parent,score_topic,nb_comments_parent,nb_comments_topic,ratio_topic,author_mean_score,isSameAuthor,year_creation_author,has_verified_email,comment_karma
0,3.0,t1_cqug90j,1,0,0,0,119,59,8,14,neg,0,2,0,0,0.000138,1.312352e-03,0.000072,night,NaN,22894,NaN,817,1,322,0.86,18.702703,False,2013.0,True,181760.0
1,3.0,t1_cqug90k,0,0,0,0,48,13,2,7,neg,0,2,0,0,0.000005,2.167334e-19,0.000000,night,13205.0,13205,10800.0,10800,1443,1443,0.86,1.750000,False,2012.0,True,23937.0
2,5.0,t1_cqug90z,1,0,0,0,4,4,1,0,neu,0,2,0,0,0.000126,1.712427e-03,0.000051,night,NaN,20339,NaN,3268,5,1610,0.82,3.105263,False,2013.0,True,125197.0
3,1.0,t1_cqug91c,1,0,0,0,54,15,3,12,neg,0,2,0,0,0.000000,2.167334e-19,0.000000,night,NaN,109679,NaN,13643,4,22,0.82,1.000000,False,2013.0,True,7663.0
4,101.0,t1_cqug91e,1,0,0,0,241,164,24,20,pos,0,2,1,2,0.000004,5.092290e-05,0.000002,night,NaN,22895,NaN,817,2,322,0.86,51.500000,False,2013.0,True,5344.0


In [37]:
df.shape

(4234970, 31)

Si on ne tient pas en compte des colonnes `ups` et `name`, nous avons mis au point 29 features afin d'essayer de prédire le score d'un commentaire. <br> 
Bien évidemment, à terme, toutes ne seront pas utilisées pour entrainer le modèle, nous garderons que celles qui permettent d'obtenir les meilleurs résultats.

In [38]:
# Sauvegarde DataFrame dans un fichier 
df.to_json(pathFile + "df_features.json")